# Assignment 2 : link_prediction





## Download Package

In [62]:
# Install torch geometric
!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
!pip install -q torch-geometric

In [63]:
!pip install wandb

## Import Package

In [64]:
import torch
import torch.nn as nn
from torch.nn import Linear
import csv
import numpy as np
from torch_geometric.data import Data
import wandb
from sklearn.metrics import f1_score
from torch_geometric.nn import GCNConv
import os
from pathlib import Path
import pandas as pd 


## dataset 

In [65]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [78]:
d = 2
train1_path = f"/content/drive/My Drive/dataset/dataset{d}/train.csv"
test1_path = f"/content/drive/My Drive/dataset/dataset{d}/test.csv"
content_path = f"/content/drive/My Drive/dataset/dataset{d}/content.csv"
epocht= 5000
config = {
  "dataset": "follower-followee",
  "model": "GNN",
  "learning_rate": 0.001,
}
config = wandb.config
config.learning_rate = 0.01


In [79]:
import sys

class load_data(): 
  def __init__(self, path, attr_path):
    super(load_data,self).__init__()    
    self.path = path
    self.attr_path = attr_path

  def read_data(self):
      file = pd.read_csv(self.path)
      label = file.loc[:,'label'].values
      bilabel = []
      bilabel.extend(label)
      bilabel.extend(label)
      bilabel= np.array(bilabel)
      edgeto = file.loc[:,'to'].values
      edgefrom = file.loc[:,'from'].values

      edge_index = [[],[]]
      pos_edge_index = [[],[]]
      non_zero = np.nonzero(np.array(label))[0]

      edge_index[0].extend(edgeto)
      edge_index[1].extend(edgefrom)
      edge_index[1].extend(edgeto)
      edge_index[0].extend(edgefrom)
      edge_index = torch.tensor(edge_index)
      # positive
      pos_edge_index[0].extend(edgeto[non_zero])
      pos_edge_index[1].extend(edgefrom[non_zero])
      pos_edge_index[1].extend(edgeto[non_zero])
      pos_edge_index[0].extend(edgefrom[non_zero])
      pos_edge_index = torch.tensor(pos_edge_index)

      return edge_index, pos_edge_index, bilabel


  def read_content(self):
      attribution = []; node=[]
      content = pd.read_csv(self.attr_path,delimiter='\t',header=None)
      print(np.shape(content))
      contentArray = content.values
      attribute = contentArray[contentArray[:, 0].argsort()]
      print(contentArray[:, 0].argsort())
      np.set_printoptions(threshold=sys.maxsize)
      print(contentArray[360])
      for i in attribute:
        n = i[0]
        x = i[1:]
        attribution.append(x)
        node.append(n)
      attribution = torch.tensor(attribution, dtype= torch.float)
      node_num = np.shape(node)[0]
      return attribution, node_num


# Model

In [80]:

class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)


    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)

    def decode(self, z, edge_label_index):
        # cosine similarity
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim = -1)

    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple = False).t()


In [81]:

Dt = load_data(train1_path,content_path)
edge_index, pos_edge,  label = Dt.read_data()
attribution, node_num = Dt.read_content()
data = Data(x=attribution, edge_index = edge_index, pos_edge_index=pos_edge, y =label, num_nodes = node_num)

print(data)
print("edge_index",edge_index)
print("label",label)
print("pos_edge",pos_edge)
print("node_num",node_num)
attr = data.x


(3312, 3704)
[  72 2332  405 1324  783 2187 1684 1762 3184  268 2170 1104 2347  727
   65 2270 1107 1582  671 1001 2968 1206  189  174  533 1473 1939  358
 3215 1498  814 2991 3162  680 1480 1330    5 1730 1262  794 2491 1012
 1199 1620 2671   90 1287  857 1949 1058 1126 2166  603 2407 2810 2105
  475 2415  280 1093  510 1311 2703 1962 2766  235 2555  177 2451  630
 2672 3300 1810  946 1200 2387 2886  839 1239 2774 2214 2251 3222 2269
 1975  779 1605 2704 1216 2714 2549  120 1845 2660  316 3068 2473 3218
 1470 1332 1960   69 2071 1214 1657 2447 1285 3179 1507 1835  546 2179
 2471  823 2235  645 1639 2203 2461  918 2959 1685 1401 3079  381 2502
 1225 2574  926  571 1808 3239 2833  206 2650 1703  690  986  369  485
 2907 3116 2385 1695 1228 1989   11 2796  318  196  348  672 2360  648
 2417 1388 2939  412 3283  941  620 1712 2526 2630 1120 3017 2614  905
 2885 2920 2656 1723 2634  477  502 3036  303 1315  762  773  180  300
 2322  157 2992 2241 3272 2338 2625 2246 1954 2262 2282 2973 239

In [82]:
in_channel = np.shape(attr[0])[0]
model = Net(in_channel, 128, 1)
optimizer = torch.optim.Adam(params = model.parameters(), lr=0.001, weight_decay=5e-4)
criterion = torch.nn.BCEWithLogitsLoss()

In [83]:
total_examples = total_loss = 0
wandb.watch(model)
wandb.init(project="Link Prediction") 

def train():
    threshold = torch.tensor([0.4])
    min_valid_loss = 1
    model.train()
    for epoch in range(epocht):
          optimizer.zero_grad()
          z = model.encode(data.x, data.pos_edge_index)
          out = model.decode(z, data.edge_index).view(-1)
          s_predict = out.sigmoid()
          loss = criterion(out, torch.FloatTensor(data.y))
          predicted = (s_predict > threshold).float()
          f1score = f1_score(predicted.detach().numpy(), data.y.astype(float))
          loss.backward()
          optimizer.step()
          if epoch % 100 == 0:
            wandb.log({'pair_loss': loss, 'epoch': epoch})
            wandb.log({'f1score': f1score, 'epoch': epoch})
            print(f'epoch: {epoch} / training loss: {loss} /  f1score {f1score}')
            if min_valid_loss > loss:
              min_valid_loss = loss
            # Saving State Dict
              torch.save(model.state_dict(), '/content/drive/My Drive/dataset/saved_model.pth')

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
f1score,▁▇▇████████████████████▇████████████████
pair_loss,█▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,4900
f1score,0.78915
pair_loss,0.33341


In [ ]:
train()

epoch: 0 / training loss: 0.6922793388366699 /  f1score 0.6624113475177306
epoch: 100 / training loss: 0.35674411058425903 /  f1score 0.821408701457661
epoch: 200 / training loss: 0.32892706990242004 /  f1score 0.8296734554556834
epoch: 300 / training loss: 0.3187797963619232 /  f1score 0.8326465201465201
epoch: 400 / training loss: 0.3153163194656372 /  f1score 0.8327989005955109
epoch: 500 / training loss: 0.3102813959121704 /  f1score 0.835081249279705
epoch: 600 / training loss: 0.30897262692451477 /  f1score 0.8346764909049045
epoch: 700 / training loss: 0.30664610862731934 /  f1score 0.8354371742906775
epoch: 800 / training loss: 0.3047609329223633 /  f1score 0.834392696174737


## Testing 

In [ ]:

class load_data():
    def __init__(self, ):
        super(load_data, self).__init__()
        self.data = {}

    def read_data(self,path):
        file = pd.read_csv(path)
        label = file.loc[:, 'label'].values
        edgeto = file.loc[:, 'to'].values
        edgefrom = file.loc[:, 'from'].values
        pos_edge_index = [[], []]
        non_zero = np.nonzero(np.array(label))[0]
        # positive
        pos_edge_index[0].extend(edgeto[non_zero])
        pos_edge_index[1].extend(edgefrom[non_zero])
        pos_edge_index[1].extend(edgeto[non_zero])
        pos_edge_index[0].extend(edgefrom[non_zero])
        edge_index = torch.tensor(pos_edge_index)
        return edge_index

    def read_test_data(self,path):
        file = pd.read_csv(path)
        edg_id = file.loc[:, 'id'].values
        edgeto = file.loc[:, 'to'].values
        edgefrom = file.loc[:, 'from'].values
        edge_index = [[], []]
        edge_index[0].extend(edgeto)
        edge_index[1].extend(edgefrom)
        test_edge_index = torch.tensor(edge_index)
        return test_edge_index, edg_id

    def read_content(self,path):
        attribution = [];
        node = []
        content = pd.read_csv(path, delimiter='\t', header=None)
        contentArray = content.values
        attribute = contentArray[contentArray[:, 0].argsort()]
        for i in attribute:
            n = i[0]
            x = i[1:]
            attribution.append(x)
            node.append(n)
        attribution = torch.tensor(attribution, dtype=torch.float)
        node_num = np.shape(node)[0]
        return attribution, node_num



In [ ]:
Dt = load_data()
pos_edge = Dt.read_data(train1_path)
edge_index_test, eid_test = Dt.read_test_data(test1_path)
attribution, node_num = Dt.read_content(content_path)

data = Data(x=attribution, edge_index=pos_edge, edge_index_test = edge_index_test, edge_id = eid_test )
edge_index= data.edge_index
attr = data.x
edge_index_test = data.edge_index_test
edge_id = data.edge_id


In [ ]:
in_channel = np.shape(attr[0])[0]
model = Net(in_channel, 128, 1)
model_path = torch.load('/content/drive/My Drive/dataset/saved_model.pth')
model.load_state_dict(model_path)

In [ ]:
@torch.no_grad()
def test():
    model.eval()
    scores = []; eid = []
    z = model.encode(attr, edge_index)
    out = model.decode(z, edge_index_test).view(-1).sigmoid()
    scores.extend(out.numpy())
    eid.extend(np.array(edge_id))
    return scores, eid


In [ ]:
if __name__ == '__main__':
    score, eid = test()
    result = {"id": eid,
              "prob":score}
    result = pd.DataFrame( result)
    result.to_csv(f'result{d}.csv',index=False)